# Notes on Phase Diversity

# Instrument model

The image intensity $i$ of an object of intensity $o$ is obtained by convolution with the *point spread function* (PSF) $h$:

$$\begin{align}
i(x, y) &= h(x, y, \lambda) * o(x, y, \lambda) + \epsilon(x, y) \,,
\end{align}$$

where $\epsilon$ is some noise.

In the case of a point source at position $(x_0, y_0)$ with spectrum $S$ (e.g. a star):

$$\begin{align}
o(x, y, \lambda) &= S(\lambda) \, \delta_{x_0, y_0}(x, y) \,,
\end{align}$$

the system can be modeled with a so-called *broadband* PSF $h_S$, which results from the integration of monochromatic PSFs $h_\lambda$'s weighted by the object spectrum:

$$\begin{align}
i(x, y) &= h_S(x, y) * \delta_{x_0, y_0}(x, y) + \epsilon(x, y) \\
\text{with } h_S(x, y) &= \int_\lambda S(\lambda) \, h_\lambda(x, y) \, \text{d}\lambda \,.
\end{align}$$

The monochromatic PSF derives from several monochromatic components -- optics, detector, attitude and orbit -- modeled as PSFs.
The monochromatic PSF is the convolution of those.

The monochromatic *optical* PSF at wavelengh $\lambda$, denoted $h^\text{opt}_\lambda$,
is computed from the inverse Fourier transform of the *complex pupil function* (CPF).
The CPF is obtained from the binary pupil mask, $P$, and the *wave front error* (WFE) or *phase*, $\Phi$,
which is itself modeled as a series of Zernike polynomials:

$$\begin{align}
h^\text{opt}_\lambda(x, y) &= |\mathcal{F}^{-1}[P \, \exp(-j2\pi/\lambda\cdot\Phi)](x, y)|^2 \\
\text{with } \Phi(u, v) &= \sum_k \alpha_k Z_k(u, v) \,,
\end{align}$$

where $(x, y)$ denotes image plane coordinates, while $(u, v)$ denotes puil plane coordinates.
The $\alpha_k$'s are the *Zernike coefficients* which we want to estimate, and $Z$ is the Zernike basis.
Note that $\Phi$ is wavelength-independent.

The monochromatic system PSF $h_\lambda$ is obtained by convolving the monochromatic optical PSF with other PSFs,
namely the detector PSF, $h_\lambda^\text{det}$, and the attitude and orbit (AO) PSF which arises from the motion of the satellite, $h_\lambda^\text{ao}$:

$$\begin{align}
h_\lambda(x, y) &= (h^\text{opt}_\lambda * h^\text{det}_\lambda * h^\text{ao}_\lambda)(x, y) \,.
\end{align}$$

They are both modeled as Gaussian PSFs of respective standard deviations $\sigma_\lambda^\text{det}$ and $\sigma_\lambda^\text{ao}$,
which are characterized a priori.

# Phase diversity

Phase retrieval from phase diversity consists in estimating the Zernike coefficients $\alpha_k$'s from a set of $N$ images $i_1, i_2, \,...\, , i_N$ acquired with varying positions of the M2 mirror (aka varying defocus).
Defocusing is modeled as a variation of $\alpha_4$ and $\alpha_{11}$ (using Noll's indexing), all other Zernike coefficients being equal.

For $n = 1 \,..\, N$, let us consider an image of one star of spectrum $S_n$ and position $(x_n, y_n)$.
Then:

$$\begin{align}
i_n(x, y) &= a_n \, h_{S_n}(x, y) + b_n + \epsilon(x, y)\,,
\end{align}$$

where $a_n$ and $b_n$ are some normalization parameters which simplify computation.

The following error function is introduced, assuming a Gaussian noise:

$$\begin{align}
E &= \sum_n \sum_{(x,y)} \frac{\| a_n \, h_{S_n}(x, y) + b_n - i_n(x, y)\|^2}{\sigma_n^2} \,,
\end{align}$$

where $\sigma_n^2$ is the noise variance in the image.

The parameters of $E$ -- the unknowns -- are:
* The $\alpha_k$'s for $k \neq 4, 11$ which do not depend on the image;
* For each image, the position of the stars $(x_n, y_n)$;
* For each image, the fourth and eleventh Zernike coefficients, denoted $\alpha_{4,n}$ and $\alpha_{11,n}$, which are the only coefficients which differ from one image to another;
* For each image, the normalization parameters $a_n$ and $b_n$.


# Numerical computation

## Monochromatic optical PSF

The pupil mask $P$ is a square image essentially made of zeros and ones, except at edges, where it is smoothed for anti-aliasing purpose.
For Euclid, the mask is of side $1024$, and the pupil of side $512$.

The Zernike basis is precomputed up to the 8-th order, thus resulting in a $512 \times 512$ map of vectors of $45$ values,
stored padded as a $45 \times 1024 \times 1024$ cube in order to ensure data locality in memory throughout expensive computations.

The complex exponential is effectively computed only where $P$ is non null.
This should make the the padding negligible in terms of computation time.

At wavelength $\lambda \, \text{nm}$, Each PSF pixel spans over $\lambda / 1024 \, \text{nm}$.
Euclid wavelengths span from $500$ to $900\,\text{nm}$, and therefore a pixel measures roughly $0.5$ to $0.9\,\text{nm}$.

Given that wavelengths are processed in sequence, the same memory locations are reused.

## Monochromatic system TF

At system level, instead of working with PSFs, transfer functions (TFs) are used, which allows performing convolutions faster and improves the quality of the spectral integration.
They are Fourier transforms of the PSFs.
Although $\sigma_\text{det}$ and $\sigma_\text{ao}$ are wavelength-independent, the PSF coordinate system is, which means that $h^\text{det}(x, y)$ and $h^\text{ao}(x, y)$ depend on $\lambda$ via $(x, y)$

For each $\lambda$ and each $(x, y)$, the product of the detector and AO TFs, $H^\text{det}(x, y) \times H^\text{ao}(x, y)$, is precomputed.

Similarly to PSFs, each monochromatic TF reuses the memory locations of the previous $\lambda$.

## Broadband PSF

The broadband PSF is computed as the inverse Fourier transform of the broadband TF,
that is, the integration over $\lambda$ is performed on the TFs,
and the result is later transformed back into a PSF.

To this end, monochromatic TFs are first warped to a common grid.
The size of the grid is determined by the desired size of the PSF in the image domain, multiplied by some oversampling factor.
As of today, the size is set to $100\,\text{px}$ and the factor to $3$, which gives a $300 \times 300$ PSF stamp,
as well as a $40 \times 300 \times 300$ TF cube (where $40$ is the number of wavelengths).

Once warped, the monochromatic system TFs are interpolated along the $\lambda$ axis for a more precise integration.
The TF cube is resampled at $2\,\text{nm}$, would output a $200 \times 300 \times 300$ cube.
Yet, since the only operation to be performed on this TF cube would be a numerical integration along $\lambda$,
the cube is not instantiated.
Instead, spline coefficients are estimated, and the integral of the spline is computed directly,
without computing the interpolated values (more on this to follow).
This way, the transform from monochromatic system TFs to broadband TF is direct.

## On the interpolation and integration along wavelengths

TODO

## Minimization

TODO